<a href="https://colab.research.google.com/github/lipeng2021/-python1/blob/main/%E4%B8%A2%E5%BC%83%E6%B3%95-%E5%87%BD%E6%95%B0%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
%matplotlib inline
import torch
import IPython as display
import torch.nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import sys
import time
import torch.nn as nn
import torch.optim as optim
FILENAME = './content/sample_data/lp'

In [69]:
def dropout(x,drop_prob):
  x = x.float()
  assert 0<= drop_prob <=1
  keep_prob = 1-drop_prob
  if keep_prob==0:
    return torch.zeros_like(x)
  mask = (torch.randn(x.shape)<keep_prob).float()
  return mask*x/keep_prob

In [70]:
x = torch.arange(16).view(2,8)
dropout(x,0)

tensor([[ 0.,  1.,  2.,  3.,  0.,  5.,  6.,  7.],
        [ 8.,  9., 10.,  0.,  0., 13., 14., 15.]])

初始化参数

In [71]:
num_inputs =784
num_outputs=10
num_hiddens1 = 256
num_hiddens2=256
w1 = torch.tensor(np.random.normal(0,0.01,size=(num_inputs,num_hiddens1)),dtype=torch.float,requires_grad=True)
w2 = torch.tensor(np.random.normal(0,0.01,size=(num_hiddens1,num_hiddens2)),dtype=torch.float,requires_grad=True)
w3 = torch.tensor(np.random.normal(0,0.01,size=(num_hiddens2,num_outputs)),dtype=torch.float,requires_grad=True)
b1 = torch.zeros(num_hiddens1,requires_grad=True)
b2 = torch.zeros(num_hiddens2,requires_grad=True)
b3 = torch.zeros(num_outputs,requires_grad=True)
params = [w1,b1,w2,b2,w3,b3]

In [72]:
def relu(x):
  return torch.max(input=x,other=(0,0))

In [87]:
drop_prob1 = 0.2
drop_prob2 = 0.5
def net(x,is_training=True):
  x = x.view(-1,num_inputs)
  h1 = torch.matmul(x,w1)+b1
  h1 = h1.relu()
  if is_training:
    h1=dropout(h1,drop_prob1)
  h2 = torch.matmul(h1,w2)+b2
  h2 = h2.relu()
  if is_training:
    h2 = dropout(h2,drop_prob2)
  h3 = torch.matmul(h2,w3)+b3
  return h3


模型评估

In [74]:
def evaculate_accuracy(data_iter,net):
  acc_sum = 0.0
  num=0
  for X,y in data_iter:
    if isinstance(net,torch.nn.Module):
      net.eval()
      acc_sum += (net(X).argmax(dim=1)==y).float().sum().item()
      net.train()
    else:
      if('is_training' in net.__code__.co_varnames):
        acc_sum += (net(X,is_training=False).argmax(dim=1)==y).float().sum().item()
      else:
        acc_sum +=(net(X).argmax(dim=1)==y).float().sum().item()
    num = y.shape[0]
    return acc_sum/num

In [75]:
mnist_train = torchvision.datasets.FashionMNIST(root=FILENAME,train=True,transform=transforms.ToTensor(),download=True)
mnist_test = torchvision.datasets.FashionMNIST(root=FILENAME,train=False,transform=transforms.ToTensor(),download=True)

读取数据

In [76]:
batch_size = 256
if sys.platform.startswith('win'):
  num_worker = 0
else:
  num_workers = 2
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=num_workers)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=True,num_workers=num_workers)

定义损失函数

In [77]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(params,lr=0.5)
num_epochs = 5

模型训练

In [90]:
def train_dropout(net,train_iter,test_iter,batch_size,num_epochs,loss,params=None,optimizer=None):
  for epoch in range(num_epochs):
    train_ls=0.0
    train_acc_sum=0.0
    test_acc_sum = 0.0
    n=0
    for X,y in train_iter:
      y_hat = net(X)
      l = loss(y_hat,y).sum()
      optimizer.zero_grad()
      l.backward()
      optimizer.step()
      train_ls += l.item()
      train_acc_sum += (net(X).argmax(dim=1)==y).float().sum().item()
      n = y.shape[0]
      test_acc_sum = evaculate_accuracy(test_iter,net)
    print('epoch%d,loss%.4f,trainacc %.3f,test acc %.3f'%(epoch+1,train_ls/n,train_acc_sum/n,test_acc_sum))

In [ ]:
train_dropout(net,train_iter,test_iter,batch_size,num_epochs,loss,params,optimizer)